# 9. Import a time series

In this tutorial we are adding a timeseries to the DataStore object. This might be useful if the temperature in one of the calibration baths was measured with an external device. It requires three steps to add the measurement files to the DataStore object:
1. Load the measurement files (e.g., csv, txt) with pandas into a pandas.Series object
2. Add the pandas.Series object to the DataStore
3. Align the time to that of the DTS measurement (required for calibration)

In [ ]:
import pandas as pd
import os

from dtscalibration import read_silixa_files

## Step 1: load the measurement files

In [ ]:
filepath = os.path.join('..', '..', 'tests', 'data', 
                        'external_temperature_timeseries', 
                        'Loodswaternet2018-03-28 02h.csv')

# Bonus:
print(filepath, '\n')
with open(filepath, 'r') as f:
    head = [next(f) for _ in range(5)]
print(' '.join(head))

In [ ]:
ts = pd.read_csv(filepath, sep=',', index_col=0, parse_dates=True, 
                 squeeze=True, engine='python')  # the latter 2 kwargs are to ensure a pd.Series is returned
ts = ts.tz_localize('Europe/Amsterdam')  # set the timezone

In [ ]:
ts.head()  # Double check the timezone

Now we quickly create a DataStore from xml-files with Stokes measurements to add the external timeseries to

In [ ]:
filepath_ds = os.path.join('..', '..', 'tests', 'data', 'double_ended2')
ds = read_silixa_files(directory=filepath_ds,
                       timezone_netcdf='UTC',
                       file_ext='*.xml')

## Step 2: Add the temperature measurements of the external probe to the DataStore.

First add the coordinates

In [ ]:
ds.coords['time_external'] = ts.index.values

Second we add the measured values

In [ ]:
ds['external_probe'] = (('time_external',), ts)

## Step 3: Align the time of the external measurements to the Stokes measurement times
We linearly interpolate the measurements of the external sensor to the times we have DTS measurements

In [ ]:
ds['external_probe_dts'] = ds['external_probe'].interp(time_external=ds.time)

In [ ]:
print(ds.data_vars)

Now we can use `external_probe_dts` when we define sections and use it for calibration